In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the embedding model (this converts text to vectors)
print("Loading model... (first time may take 1-2 minutes to download)")
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

print("✓ Model loaded successfully!")
print(f"✓ Embedding dimension: {model.get_sentence_embedding_dimension()}")

Loading model... (first time may take 1-2 minutes to download)
✓ Model loaded successfully!
✓ Embedding dimension: 384


In [3]:
# Test with a sample product description
product_description = "WHITE HANGING HEART T-LIGHT HOLDER"

# Convert text to vector (embedding)
embedding = model.encode(product_description)

print(f"Product: {product_description}")
print(f"\nVector shape: {embedding.shape}")
print(f"First 10 numbers of the vector: {embedding[:10]}")
print(f"\nThis is how the product is stored in Pinecone!")

Product: WHITE HANGING HEART T-LIGHT HOLDER

Vector shape: (384,)
First 10 numbers of the vector: [ 0.01664448  0.126716    0.02079196  0.05034028 -0.06356651 -0.02135354
  0.04959639  0.03494158  0.07634635 -0.00453511]

This is how the product is stored in Pinecone!
